In [1]:
from helpers import plotframes, plotframes_tensorboard
from mvt import MaskedVideoTransformer
from gameds import GameDS
import torch
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import random
import argparse
from dual_logger import DualLogger
import matplotlib.pyplot as plt
import uuid

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:


class Args:
    def __init__(self):
        self.device = "cpu"
        self.lr = 0.01
        self.n_frames = 2

args = Args()

writer = DualLogger(log_dir=f'./logs3/{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}_{str(uuid.uuid4())}', project_name='my_project')

args_str = "\n".join(f"{k}: {v}" for k, v in vars(args).items())
writer.tensorboard_writer.add_text('Hyperparameters', args_str, 0)



from torch.utils.data import DataLoader

model = MaskedVideoTransformer(NUM_FRAMES=args.n_frames, COLOR_CHANNELS=1, D_DIM=736, PATCH_SIZE=32)
model.to(args.device);

import wandb
from torch.utils.tensorboard import SummaryWriter

print("run started")



from PIL import Image
from helads import HelaData

hela_train = HelaData("/home/constantin/Documents/celltracking/HeLa_dataset/train", sequence_length=args.n_frames)
hela_val = HelaData("/home/constantin/Documents/celltracking/HeLa_dataset/train", sequence_length=args.n_frames)
train_dataloader = DataLoader(hela_train, batch_size=24, shuffle=True, prefetch_factor=None, num_workers=0)
val_dataloader = DataLoader(hela_val, batch_size=24, shuffle=True, prefetch_factor=None, num_workers=0)

print("Dataloader length", len(train_dataloader))

criterion = torch.nn.MSELoss()
optim = torch.optim.Adam(params=model.parameters(), lr=args.lr)

step = 0
optim.zero_grad()
for _ in range(1000000):
    torch.save(model.state_dict(), "model.pt")
    for i_step, (X, _) in enumerate(train_dataloader):

        X = X.to(args.device)
        
        
        X_pred, (X_masked, ) = model(X)
        
        print("SHAPES", X_pred.shape, X_masked.shape)

        loss = criterion(X_pred, X)
    
        loss.backward()
        if step%20 == 19 or True:
            optim.step()
            optim.zero_grad()
 
        writer.add_scalar('Training loss', loss.item(), step)
    
        if step%20 == 10:
            image_tensor = plotframes_tensorboard(X_masked, f"Train: Mask {step}")
            writer.add_image('Train: Mask', image_tensor, step)

            image_tensor = plotframes_tensorboard(X_pred, f"Train: Prediction {step}")
            writer.add_image('Train: Prediction', image_tensor, step)

            image_tensor = plotframes_tensorboard(X, f"Train: Original {step}")
            writer.add_image('Train: Original', image_tensor, step)

            image_tensor = plotframes_tensorboard([X, X_pred, X_masked], f"Train: Combined {step}")
            writer.add_image('Train: Combined', image_tensor, step)
    
            # evaluation dataset
            X_eval, _ = random.choice(hela_val)
            X_eval = X_eval[None].to(args.device)
            X_eval_pred, (X_eval_pred_masked, ) = model(X_eval)
            
            print("before eval", X_eval.shape, X_eval_pred_masked.shape, X_eval_pred.shape)
            loss = criterion(X_eval_pred, X_eval[0])
            writer.add_scalar('Validation loss', loss.item(), step)
    
            image_tensor = plotframes_tensorboard(X_eval_pred_masked, f"Val: Mask {step}")
            writer.add_image('Val: Mask', image_tensor, step)
            
            image_tensor = plotframes_tensorboard(X_eval_pred, f"Val: Prediction {step}")
            writer.add_image('Val: Prediction', image_tensor, step)
        
            image_tensor = plotframes_tensorboard(X_eval, f"Val: Original {step}")
            writer.add_image('Val: Original', image_tensor, step)

            image_tensor = plotframes_tensorboard([X_eval, X_eval_pred, X_eval_pred_masked], f"Val: Combined {step}")
            writer.add_image('Eval: Combined', image_tensor, step)

            del X_eval_pred
            del X_eval_pred_masked


        del X_pred
        del X_masked
        step += 1
    



run started
Dataloader length 298
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
SHAPES torch.Size([24, 2, 1, 128, 128]) torch.Size([24, 2, 1, 128, 128])
